In [1]:
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
from requests import get
h2o.init()
if not H2ODeepWaterEstimator.available(): exit

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,9 mins 49 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,19 minutes
H2O cluster name:,arno
H2O cluster total nodes:,1
H2O cluster free memory:,12.98 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [2]:
print("Downloading the model")
## http://data.dmlc.ml/mxnet/models/imagenet/inception-bn_old.tar.gz
## !gunzip ...

print("Importing the model architecture for scoring in H2O")
model = H2ODeepWaterEstimator(epochs=0, ## no training - just load the state - NOTE: training for this 3-class problem wouldn't work since the model has 1k classes
                            mini_batch_size=32, ## mini-batch size is used for scoring
                            ## all parameters below are needed
                            network='user',
                            network_definition_file=os.getcwd() + "/Inception_BN-symbol.json",
                            network_parameters_file=os.getcwd() + "/Inception_BN-0039.params",
                            mean_image_file=        os.getcwd() + "/mean_224.nd",
                            image_shape=[224,224],
                            channels=3
)

frame = h2o.import_file("bigdata/laptop/deepwater/imagenet/cat_dog_mouse.csv")
print(frame.head(5))
nclasses = frame[1].nlevels()[0]

model.train(x=[0],y=1, training_frame=frame) ## must call train() to initialize the model, but it isn't training

Importing the model architecture for scoring in H2O
Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2
bigdata/laptop/deepwater/imagenet/cat/102194502_49f003abd9.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/11146807_00a5f35255.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/1140846215_70e326f868.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/114170569_6cbdf4bbdb.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/1217664848_de4c7fc296.jpg,cat



deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [3]:
## Extract deep features from final layer before going into Softmax.
extracted_features = model.deepfeatures(frame, "global_pool_output")
#extracted_features = model.deepfeatures(frame, "conv_5b_double_3x3_1_output")

print(extracted_features)
assert extracted_features.ncol == 1024

deepfeatures progress: |██████████████████████████████████████████████████| 100%


DF.global_pool_output.C1 DF.global_pool_output.C2 DF.global_pool_output.C3 DF.global_pool_output.C4 DF.global_pool_output.C5 DF.global_pool_output.C6 DF.global_pool_output.C7 DF.global_pool_output.C8 DF.global_pool_output.C9 DF.global_pool_output.C10 DF.global_pool_output.C11 DF.global_pool_output.C12 DF.global_pool_output.C13 DF.global_pool_output.C14 DF.global_pool_output.C15 DF.global_pool_output.C16 DF.global_pool_output.C17 DF.global_pool_output.C18 DF.global_pool_output.C19 DF.global_pool_output.C20 DF.global_pool_output.C21 DF.global_pool_output.C22 DF.global_pool_output.C23 DF.global_pool_output.C24 DF.global_pool_output.C25 DF.global_pool_output.C26 DF.global_pool_output.C27 DF.global_pool_output.C28 DF.global_pool_output.C29 DF.global_pool_output.C30 DF.global_pool_output.C31 DF.global_pool_output.C32 DF.global_pool_output.C33 DF.global_pool_output.C34 DF.global_pool_output.C35 DF.global_pool_output.C36 DF.global_pool_output.C37 DF.global_pool_output.C38 DF.global_pool_output.C39 DF.global_pool_output.C40 DF.global_pool_output.C41 DF.global_pool_output.C42 DF.global_pool_output.C43 DF.global_pool_output.C44 DF.global_pool_output.C45 DF.global_pool_output.C46 DF.global_pool_output.C47 DF.global_pool_output.C48 DF.global_pool_output.C49 DF.global_pool_output.C50 DF.global_pool_output.C51 DF.global_pool_output.C52 DF.global_pool_output.C53 DF.global_pool_output.C54 DF.global_pool_output.C55 DF.global_pool_output.C56 DF.global_pool_output.C57 DF.global_pool_output.C58 DF.global_pool_output.C59 DF.global_pool_output.C60 DF.global_pool_output.C61 DF.global_pool_output.C62 DF.global_pool_output.C63 DF.global_pool_output.C64 DF.global_pool_output.C65 DF.global_pool_output.C66 DF.global_pool_output.C67 DF.global_pool_output.C68 DF.global_pool_output.C69 DF.global_pool_output.C70 DF.global_pool_output.C71 DF.global_pool_output.C72 DF.global_pool_output.C73 DF.global_pool_output.C74 DF.global_pool_output.C75 DF.global_pool_output.C76 DF.global_pool_output.C77 DF.global_pool_output.C78 DF.global_pool_output.C79 DF.global_pool_output.C80 DF.global_pool_output.C81 DF.global_pool_output.C82 DF.global_pool_output.C83 DF.global_pool_output.C84 DF.global_pool_output.C85 DF.global_pool_output.C86 DF.global_pool_output.C87 DF.global_pool_output.C88 DF.global_pool_output.C89 DF.global_pool_output.C90 DF.global_pool_output.C91 DF.global_pool_output.C92 DF.global_pool_output.C93 DF.global_pool_output.C94 DF.global_pool_output.C95 DF.global_pool_output.C96 DF.global_pool_output.C97 DF.global_pool_output.C98 DF.global_pool_output.C99 DF.global_pool_output.C100 DF.global_pool_output.C101 DF.global_pool_output.C102 DF.global_pool_output.C103 DF.global_pool_output.C104 DF.global_pool_output.C105 DF.global_pool_output.C106 DF.global_pool_output.C107 DF.global_pool_output.C108 DF.global_pool_output.C109 DF.global_pool_output.C110 DF.global_pool_output.C111 DF.global_pool_output.C112 DF.global_pool_output.C113 DF.global_pool_output.C114 DF.global_pool_output.C115 DF.global_pool_output.C116 DF.global_pool_output.C117 DF.global_pool_output.C118 DF.global_pool_output.C119 DF.global_pool_output.C120 DF.global_pool_output.C121 DF.global_pool_output.C122 DF.global_pool_output.C123 DF.global_pool_output.C124 DF.global_pool_output.C125 DF.global_pool_output.C126 DF.global_pool_output.C127 DF.global_pool_output.C128 DF.global_pool_output.C129 DF.global_pool_output.C130 DF.global_pool_output.C131 DF.global_pool_output.C132 DF.global_pool_output.C133 DF.global_pool_output.C134 DF.global_pool_output.C135 DF.global_pool_output.C136 DF.global_pool_output.C137 DF.global_pool_output.C138 DF.global_pool_output.C139 DF.global_pool_output.C140 DF.global_pool_output.C141 DF.global_pool_output.C142 DF.global_pool_output.C143 DF.global_pool_output.C144 DF.global_pool_output.C145 DF.global_pool_output.C146 DF.global_pool_output.C147 DF.global_pool_output.C148 DF.global_pool_output.C149 DF.global_pool_output.C150 DF.global_pool_output.C151 DF.global_pool_output.C152 DF.g

In [4]:
## Find the squared cosine similarity 
## between the first 5 images (queries) and the rest (references)
references = extracted_features[5:,:]  ## large
queries    = extracted_features[:5,:]  ## small

df = references.distance(queries, "cosine")

print(df)
assert df.shape[0] == 262
assert df.shape[1] == 5

C1,C2,C3,C4,C5
0.000700166,0.000890456,0.00115243,0.000593065,0.000788263
0.000714772,0.000971895,0.00114015,0.000604227,0.000790366
0.000725556,0.00088677,0.00108941,0.000697152,0.00082574
0.000583118,0.000677621,0.000848235,0.000597934,0.000761433
0.000709113,0.00075652,0.000968125,0.000586435,0.000784854
0.000779529,0.00103488,0.00124044,0.000713327,0.000997447
0.000725078,0.00103037,0.00122527,0.000590141,0.000993968
0.00077362,0.000987806,0.00126681,0.000670355,0.00104349
0.000733625,0.000879774,0.00120423,0.000891323,0.000960142
0.000823687,0.000976036,0.00123983,0.000840577,0.00102143
